# Tareas
1) Desde un notebook, script para llamar a ecobici y guardar alguna info SIMPLE (ej: cuántos slots ocupados) en csv.
2) Agregar el secret para usar en github actions
3) Configurar la Github Action para que haga un commit (en una branch en particular? O en main?)
4) Luego de probar que funciona, agregar el cron a la github action
5) Mergear a main el nuevo código

In [1]:
import os
from datetime import datetime, UTC
import pandas as pd
import requests
from pandas.errors import EmptyDataError

In [35]:
def set_environment():
    env_path = "../.env"
    try:
        with open(env_path, "r") as f:
            for line in f:
                if '=' in line and not line.strip().startswith("#"):
                    key, value = line.strip().split('=', 1)
                    os.environ[key] = value
    except FileNotFoundError:
        return

set_environment()

In [36]:
class TransportBAClient():
    def __init__(self):
        self.get_stations_url = "https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?"
        self.get_stations_status_url = "https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?"
        self.client = requests.Session()
        clientID = os.getenv("TRANSPORT_BA_CLIENT_ID")
        clientSecret = os.getenv("TRANSPORT_BA_SECRET")
        self.query_params = {
            "client_id": clientID,
            "client_secret": clientSecret
        }
        
    def GetStations(self):
        return self.__get(self.get_stations_url)
    
    def GetStationStatus(self):
        return self.__get(self.get_stations_status_url)
    
    def __get(self, url):
        response = self.client.get(url, params=self.query_params)
        response.raise_for_status()
        data = response.json()
        return data["data"]["stations"]


In [37]:
def preprocessStations(stations_df):
    stations_df = stations_df[["station_id", "groups"]]
    return stations_df.explode("groups")

def preprocessStationsStatus(stations_status_df):
    stations_status_df["status_last_reported_utc"] = pd.to_datetime(stations_status_df["last_reported"], unit="s", utc=True)
    return station_status_df[
        ["station_id",
         "num_bikes_available", 
         "num_bikes_disabled", 
         "num_docks_available", 
         "num_docks_disabled",
         "status_last_reported_utc"
         ]
    ]

In [38]:
set_environment()

transportBAClient = TransportBAClient()
stations = transportBAClient.GetStations()
station_status = transportBAClient.GetStationStatus()

stations_df = pd.DataFrame.from_dict(stations)
station_status_df = pd.DataFrame.from_dict(station_status)

stations_df = preprocessStations(stations_df)
station_status_df = preprocessStationsStatus(station_status_df)
join = pd.merge(stations_df, station_status_df, on="station_id")

In [48]:
class StationsDAO():
    def __init__(self):
        self.filename = "../data/stations.csv"
        self.schema = [
            "job_run_at",
            "station_id",
            "groups",
            "num_bikes_available", 
            "num_bikes_disabled", 
            "num_docks_available", 
            "num_docks_disabled",
            "status_last_reported_utc"
        ]
        self.repo = CSVRepo(self.filename, self.schema)
    
    def AppendWithJobRunAt(self, new_rows):
        new_rows["job_run_at"] = datetime.now(UTC).isoformat()
        new_rows["job_run_at"] = pd.to_datetime(new_rows["job_run_at"])
        self.repo.Append(new_rows)


class CSVRepo():
    def __init__(self, table, schema):
        self.filename = table
        self.schema = schema
        return
    
    def Append(self, new_rows: pd.DataFrame):
        if set(new_rows.columns) != set(self.schema):
            raise Exception("Schema mismatch: {} != {}".format(set(self.schema), set(new_rows.columns)))
        try:
            df = pd.read_csv(self.filename)
        except (FileNotFoundError, EmptyDataError):
                df = pd.DataFrame(columns=self.schema)
        df = pd.concat([df, new_rows], ignore_index=True)
        df.to_csv(self.filename, index=False)


In [51]:
stationsDao = StationsDAO()
stationsDao.AppendWithJobRunAt(join)